In [1]:
import pandas as pd
import os
import tensorflow as tf
import numpy as np
from collections import Counter, defaultdict
from recommenders.datasets.python_splitters import python_random_split
from recommenders.utils.constants import SEED
from recommenders.utils import tf_utils
from recommenders.datasets.pandas_df_utils import user_item_pairs

2023-05-09 23:49:25.482552: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-09 23:49:25.505898: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-09 23:49:25.506376: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-09 23:49:26.028513: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
datacsv = pd.read_csv("data/new_user_recipe.csv")
datacsv.columns = ["idx", "itemID","userID", "text", "rating"]
datacsv.drop(["idx"], inplace=True, axis=1)

In [3]:
from tqdm import tqdm
import sklearn
import ast
import json

In [4]:
import re
re_l = [
    lambda x: re.sub("\d+", "", x),    
    lambda x: re.sub("\(.*\)", "", x),
    lambda x: x.replace("/ ", ""),
    lambda x: x.replace("cups", "").replace("cup", "").replace("tablespoons", "").replace("tablespoon", "").replace("teaspoons", "").replace("teaspoon", ""),
    lambda x: x.strip(" "),
    lambda x: x.strip("s")
]
ing_counter = defaultdict(int)
def loop_re(x):
    for rel in re_l:
        x = rel(x.split(",")[0])
    ing_counter[x] += 1
    return x

In [5]:
with open("finished_reviews.txt",'r') as fr:
    finished_reviews = set(ast.literal_eval("[" + fr.readline() + "]"))

e_f = {}

with open("allrecipes-recipes.json") as f:
    for i, jsonOb in tqdm(enumerate(f)):
        if i in finished_reviews:
            recipesDict = json.loads(jsonOb)
            ing = [loop_re(i) for i in recipesDict["ingredients"]]
            e_f[i] = ing

225602it [00:02, 75422.79it/s]


In [6]:
threshold = sorted([(k,v) for k,v in dict(ing_counter).items()], key = lambda x: x[1])[-len(ing_counter)//50][1]
kept_ings = set([k for k,v in dict(ing_counter).items() if v > threshold])

In [7]:
print(len(ing_counter), len(kept_ings))

8979 179


In [8]:
datacsv['ing'] = None
for i in tqdm(finished_reviews):
    r = e_f[i]
    r = [i for i in r if i in kept_ings]
    datacsv.loc[datacsv['itemID'] == i,"ing"] = r.__repr__()
datacsv['ing'] = datacsv['ing'].map(lambda x: ast.literal_eval(x) if not x is None else [])

100%|██████████| 16376/16376 [00:03<00:00, 5304.95it/s]


In [9]:
genres_encoder = sklearn.preprocessing.MultiLabelBinarizer()
datacsv["ing"] = genres_encoder.fit_transform(
    datacsv["ing"]
).tolist()
print("num ings:", len(genres_encoder.classes_))

num ings: 161


In [10]:
train, test = python_random_split(datacsv, ratio=0.75, seed=SEED)

In [11]:
items = datacsv.drop_duplicates("itemID")[["itemID", "ing"]].reset_index(drop=True)
item_feat_shape = len(items["ing"][0])
users = datacsv.drop_duplicates("userID")[["userID"]].reset_index(drop=True)
print("Total {} items and {} users in the dataset".format(len(items), len(users)))

Total 294 items and 480 users in the dataset


In [44]:
MODEL_TYPE = "wide_deep"
STEPS = 22500  # Number of batches to train
BATCH_SIZE = 4
# Wide (linear) model hyperparameters
LINEAR_OPTIMIZER = "adagrad"
LINEAR_OPTIMIZER_LR = 0.001  # Learning rate
LINEAR_L1_REG = 0.1           # Regularization rate for FtrlOptimizer
LINEAR_L2_REG = 0.0
LINEAR_MOMENTUM = 0.0         # Momentum for MomentumOptimizer or RMSPropOptimizer
# DNN model hyperparameters
DNN_OPTIMIZER = "adadelta"
DNN_OPTIMIZER_LR = 0.1
DNN_L1_REG = 0.1           # Regularization rate for FtrlOptimizer
DNN_L2_REG = 0.0
DNN_MOMENTUM = 0.0         # Momentum for MomentumOptimizer or RMSPropOptimizer
# Layer dimensions. Defined as follows to make this notebook runnable from Hyperparameter tuning services like AzureML Hyperdrive
DNN_HIDDEN_LAYER_1 = 0     # Set 0 to not use this layer
DNN_HIDDEN_LAYER_2 = 64    # Set 0 to not use this layer
DNN_HIDDEN_LAYER_3 = 128   # Set 0 to not use this layer
DNN_HIDDEN_LAYER_4 = 512   # Note, at least one layer should have nodes.
DNN_HIDDEN_UNITS = [h for h in [DNN_HIDDEN_LAYER_1, DNN_HIDDEN_LAYER_2, DNN_HIDDEN_LAYER_3, DNN_HIDDEN_LAYER_4] if h > 0]
DNN_USER_DIM = 32          # User embedding feature dimension
DNN_ITEM_DIM = 16          # Item embedding feature dimension
DNN_DROPOUT = 0.8
DNN_BATCH_NORM = 1         # 1 to use batch normalization, 0 if not.

In [45]:
import recommenders.models.wide_deep.wide_deep_utils as wide_deep

In [46]:
save_checkpoints_steps = max(1, STEPS // 5)

wide_columns, deep_columns = wide_deep.build_feature_columns(
    users=users["userID"].values,
    items=items["itemID"].values,
    user_col="userID",
    item_col="itemID",
    item_feat_col="ing",
    crossed_feat_dim=1000,
    user_dim=DNN_USER_DIM,
    item_dim=DNN_ITEM_DIM,
    item_feat_shape=item_feat_shape,
    model_type=MODEL_TYPE,
)

print("Wide feature specs:")
for c in wide_columns:
    print("\t", str(c)[:100], "...")
print("Deep feature specs:")
for c in deep_columns:
    print("\t", str(c)[:100], "...")

Wide feature specs:
	 VocabularyListCategoricalColumn(key='userID', vocabulary_list=(641, 389, 892, 1546, 1888, 1857, 2222 ...
	 VocabularyListCategoricalColumn(key='itemID', vocabulary_list=(34, 110, 113, 125, 152, 157, 199, 202 ...
	 CrossedColumn(keys=(VocabularyListCategoricalColumn(key='userID', vocabulary_list=(641, 389, 892, 15 ...
Deep feature specs:
	 EmbeddingColumn(categorical_column=VocabularyListCategoricalColumn(key='userID', vocabulary_list=(64 ...
	 EmbeddingColumn(categorical_column=VocabularyListCategoricalColumn(key='itemID', vocabulary_list=(34 ...
	 NumericColumn(key='ing', shape=(161,), default_value=None, dtype=tf.float32, normalizer_fn=None) ...


In [47]:
from tempfile import TemporaryDirectory
TMP_DIR = TemporaryDirectory()
model_dir = TMP_DIR.name

In [48]:
model = wide_deep.build_model(
    model_dir=model_dir,
    wide_columns=wide_columns,
    deep_columns=deep_columns,
    linear_optimizer=tf_utils.build_optimizer(LINEAR_OPTIMIZER, LINEAR_OPTIMIZER_LR, **{
        'l1_regularization_strength': LINEAR_L1_REG,
        'l2_regularization_strength': LINEAR_L2_REG,
        'momentum': LINEAR_MOMENTUM,
    }),
    dnn_optimizer=tf_utils.build_optimizer(DNN_OPTIMIZER, DNN_OPTIMIZER_LR, **{
        'l1_regularization_strength': DNN_L1_REG,
        'l2_regularization_strength': DNN_L2_REG,
        'momentum': DNN_MOMENTUM,  
    }),
    dnn_hidden_units=DNN_HIDDEN_UNITS,
    dnn_dropout=DNN_DROPOUT,
    dnn_batch_norm=(DNN_BATCH_NORM==1),
    log_every_n_iter=max(1, STEPS//10),  # log 10 times
    save_checkpoints_steps=save_checkpoints_steps,
    seed=SEED
)

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpwt_h1ple', '_tf_random_seed': 42, '_save_summary_steps': 100, '_save_checkpoints_steps': 4500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 2250, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [49]:
cols = {
    'col_user': "userID",
    'col_item': "itemID",
    'col_rating': "rating",
    'col_prediction': "predict",
}

# Prepare ranking evaluation set, i.e. get the cross join of all user-item pairs
ranking_pool = user_item_pairs(
    user_df=users,
    item_df=items,
    user_col="userID",
    item_col="itemID",
    user_item_filter_df=train,  # Remove seen items
    shuffle=True,
    seed=SEED
)

In [50]:
hooks = []
train_fn = tf_utils.pandas_input_fn(
    df=train,
    y_col="rating",
    batch_size=BATCH_SIZE,
    num_epochs=None,  # We use steps=TRAIN_STEPS instead.
    shuffle=True,
    seed=SEED,
)

In [ ]:
%%time
print(
    "Training steps = {}, Batch size = {} (num epochs = {})"
    .format(STEPS, BATCH_SIZE, (STEPS*BATCH_SIZE)//len(train))
)

try:
    model.train(
        input_fn=train_fn,
        hooks=hooks,
        steps=STEPS
    )
except tf.train.NanLossDuringTrainingError:
    import warnings
    warnings.warn(
        "Training stopped with NanLossDuringTrainingError. "
        "Try other optimizers, smaller batch size and/or smaller learning rate."
    )

In [52]:
import recommenders.evaluation.python_evaluation as evaluator

In [53]:
# RATING_METRICS = [
#     evaluator.map_at_k.__name__,
#     evaluator.ndcg_at_k.__name__,
#     evaluator.precision_at_k.__name__,
#     evaluator.recall_at_k.__name__,
# ]
# if len(RATING_METRICS) > 0:
#     predictions = list(model.predict(input_fn=tf_utils.pandas_input_fn(df=test)))
#     prediction_df = test.drop("rating", axis=1)
#     prediction_df["predict"] = [p['predictions'][0] for p in predictions]
    
#     rating_results = {}
#     for m in RATING_METRICS:
#         result = evaluator.metrics[m](test, prediction_df, **cols)
#         rating_results[m] = result
#     print(rating_results)

In [ ]:
predictions = list(model.predict(input_fn=tf_utils.pandas_input_fn(df=test)))
prediction_df = test.drop("rating", axis=1)
prediction_df["predict"] = [p['predictions'][0] for p in predictions]

In [55]:
prediction_df_max = prediction_df['predict'].max()
prediction_df_min = prediction_df['predict'].min()
prediction_df['predict'] = (prediction_df['predict'] - prediction_df_min) / (prediction_df_max - prediction_df_min)

In [ ]:
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
eval_map = map_at_k(test, prediction_df, col_prediction='predict', k=10)
eval_ndcg = ndcg_at_k(test, prediction_df, col_prediction='predict', k=10)
eval_precision = precision_at_k(test, prediction_df, col_prediction='predict', k=10)
eval_recall = recall_at_k(test, prediction_df, col_prediction='predict', k=10)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')